In [33]:
!pip install opencv-python


In [34]:
!pip install mediapipe



In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic#holistic model
mp_drawing = mp.solutions.drawing_utils #Drawing utilities

In [3]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#color conversion
    image.flags.writeable =False   #image is no longer writeable
    results = model.process(image) # maake prediction
    image.flags.writeable =True  #make it readable
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) # color conversion
    return image,results

In [4]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)#Draw face connection
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [5]:
def  draw_style_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color = (80,110,10),thickness =1,circle_radius = 1),
                             mp_drawing.DrawingSpec(color = (80,256,121),thickness =1,circle_radius = 1))#Draw face connection
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (80,22,10),thickness =2,circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (80,44,121),thickness =2,circle_radius = 2))
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (80,22,10),thickness =2,circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (80,44,121),thickness =2,circle_radius = 2))
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (80,117,10),thickness =2,circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (80,66,121),thickness =2,circle_radius = 2))

In [40]:
#  mp_drawing.draw_landmarks??

In [41]:
# mp_holistic.FACEMESH_TESSELATION

In [42]:
cap = cv2.VideoCapture(0)
#acess media pipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence =  0.5) as holistic:
    while cap.isOpened():
        #Read the feed
        ret,frame =cap.read()

        #make detection
        image,results = mediapipe_detection(frame,holistic)
        
        #draw landmarks
        draw_style_landmarks(image,results)
         #showw screen 
        cv2.imshow("OpenCV Frrd" , image)

        if cv2.waitKey(10) & 0xFF ==ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

In [43]:
# len(results.face_landmarks.landmark)

In [44]:
# draw_landmarks(frame,results)

In [45]:
# plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

In [46]:
# mp.drawing.draw_landmarks(image,results.face_landmarks,mp.holistic.FACE_CONNECTIONS)

In [47]:
# mp_holistic.FACEMESH_TESSELATION

Extract key point

In [6]:
pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
face = np.array([[res.x,res.y,res.z,res.visibility] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
lh= np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()  if results.left_hand_landmarks else np.zeros(21*3)
rh= np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

NameError: name 'results' is not defined

In [59]:
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh= np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()  if results.left_hand_landmarks else np.zeros(21*3)
    rh= np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [50]:
# result_test = extract_keypoints(results).shape

In [51]:
# np.save("0",result_test)

In [52]:
# len(results.left_hand_landmarks.landmark)

In [53]:
# len(results.pose_landmarks.landmark)

In [54]:
# rh= np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()

In [55]:
# lh

In [56]:
# rh

Setup for collection

In [7]:
#path for expoted data,numpy array
DATA_PATH = os.path.join("MP_Data")
#action to detect
actions = np.array(["hello","thanks","iloveyou"])
#30 videos worth of data
no_sequences = 30
#video are goinf to be 30 frames in length
sequences_length = 30

In [58]:
# we make folder 
#1 hello
## 0 frames 30 differnt values init 
##1......
###29

#2thanks

#3i love you

In [8]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))# creat folder in this action folder in that sequance folder
        except:
            pass

colllecting a data

In [62]:
cap = cv2.VideoCapture(0)
#acess media pipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence =  0.5) as holistic:
    
    #new loop
    for action in actions:
        for sequance in range(no_sequences):
            for frame_num in range(sequences_length):
                
                
                #Read the feed
                ret,frame =cap.read()

                #make detection
                image,results = mediapipe_detection(frame,holistic)

                #draw landmarks
                draw_style_landmarks(image,results)
                
                #new apply collection logic
                if frame_num == 0:
                    cv2.putText(image,"STARTING COLLECTION" , (120,200),
                               cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image,"Collection frames for {} video Number {}".format(action,sequance),(15,12),
                              cv2.FONT_HERSHEY_SIMPLEX , 0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow("OpenCV Feed" , image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,"Collection frames for {} video Number {}".format(action,sequance),(15,12),
                              cv2.FONT_HERSHEY_SIMPLEX , 0.5,(0,0,255),1,cv2.LINE_AA)
                    
                    #showw screen 
                    cv2.imshow("OpenCV Feed" , image)
                
                
                #new expoert
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH,action,str(sequance),str(frame_num))
                np.save(npy_path,keypoints)
                
                
                

                if cv2.waitKey(10) & 0xFF ==ord("q"):
                    break

    cap.release()
    cv2.destroyAllWindows()

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label : num for num ,label in enumerate(actions)}

In [11]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [12]:
sequences , labels =[] , []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequences_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
np.array(sequences).shape

(90, 30, 1662)

In [14]:
np.array(labels).shape

(90,)

In [15]:
X =np.array(sequences)

In [17]:
y= to_categorical(labels).astype(int)#use one hot encoder

In [18]:
y.shape

(90, 3)

In [27]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.05)

In [34]:
X_train.shape,X_test.shape

((85, 30, 1662), (5, 30, 1662))

In [33]:
y_train.shape

(85, 3)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [35]:
log_dir= os.path.join("Logs")
tc_callback = TensorBoard(log_dir =log_dir)

In [38]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation="relu", input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation="relu"))
model.add(LSTM(64, return_sequences=False, activation="relu"))
model.add(Dense(80, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(actions.shape[0], activation="softmax"))



In [39]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])



In [40]:
model.fit(X_train, y_train, epochs=1500, callbacks=[tc_callback])

Epoch 1/1500
3/3 [==============================] - 3s 68ms/step - loss: 4.9691 - categorical_accuracy: 0.2941
Epoch 2/1500
3/3 [==============================] - 0s 70ms/step - loss: 15.7489 - categorical_accuracy: 0.3412
Epoch 3/1500
3/3 [==============================] - 0s 67ms/step - loss: 22.9725 - categorical_accuracy: 0.3412
Epoch 4/1500
3/3 [==============================] - 0s 67ms/step - loss: 21.9794 - categorical_accuracy: 0.2588
Epoch 5/1500
3/3 [==============================] - 0s 66ms/step - loss: 23.0758 - categorical_accuracy: 0.2941
Epoch 6/1500
3/3 [==============================] - 0s 70ms/step - loss: 21.6226 - categorical_accuracy: 0.3529
Epoch 7/1500
3/3 [==============================] - 0s 71ms/step - loss: 23.7678 - categorical_accuracy: 0.3529
Epoch 8/1500
3/3 [==============================] - 0s 70ms/step - loss: 24.9381 - categorical_accuracy: 0.2824
Epoch 9/1500
3/3 [==============================] - 0s 71ms/step - loss: 23.5959 - categorical_accuracy: 

3/3 [==============================] - 0s 63ms/step - loss: 2.8602 - categorical_accuracy: 0.3176
Epoch 74/1500
3/3 [==============================] - 0s 76ms/step - loss: 4.7487 - categorical_accuracy: 0.3765
Epoch 75/1500
3/3 [==============================] - 0s 70ms/step - loss: 3.8025 - categorical_accuracy: 0.2824
Epoch 76/1500
3/3 [==============================] - 0s 66ms/step - loss: 3.4095 - categorical_accuracy: 0.4118
Epoch 77/1500
3/3 [==============================] - 0s 67ms/step - loss: 2.9507 - categorical_accuracy: 0.3765
Epoch 78/1500
3/3 [==============================] - 0s 65ms/step - loss: 2.8017 - categorical_accuracy: 0.2941
Epoch 79/1500
3/3 [==============================] - 0s 68ms/step - loss: 3.4101 - categorical_accuracy: 0.3529
Epoch 80/1500
3/3 [==============================] - 0s 71ms/step - loss: 4.7057 - categorical_accuracy: 0.2941
Epoch 81/1500
3/3 [==============================] - 0s 67ms/step - loss: 3.0036 - categorical_accuracy: 0.4118
Epoch 

3/3 [==============================] - 0s 71ms/step - loss: 2.2647 - categorical_accuracy: 0.4706
Epoch 146/1500
3/3 [==============================] - 0s 73ms/step - loss: 2.4534 - categorical_accuracy: 0.4000
Epoch 147/1500
3/3 [==============================] - 0s 73ms/step - loss: 3.9717 - categorical_accuracy: 0.3059
Epoch 148/1500
3/3 [==============================] - 0s 66ms/step - loss: 2.8887 - categorical_accuracy: 0.3412
Epoch 149/1500
3/3 [==============================] - 0s 67ms/step - loss: 2.4003 - categorical_accuracy: 0.4000
Epoch 150/1500
3/3 [==============================] - 0s 69ms/step - loss: 1.7327 - categorical_accuracy: 0.4471
Epoch 151/1500
3/3 [==============================] - 0s 71ms/step - loss: 1.7260 - categorical_accuracy: 0.4471
Epoch 152/1500
3/3 [==============================] - 0s 67ms/step - loss: 1.5887 - categorical_accuracy: 0.4353
Epoch 153/1500
3/3 [==============================] - 0s 67ms/step - loss: 1.4595 - categorical_accuracy: 0.517

3/3 [==============================] - 0s 72ms/step - loss: 1.3829 - categorical_accuracy: 0.4706
Epoch 218/1500
3/3 [==============================] - 0s 65ms/step - loss: 1.7825 - categorical_accuracy: 0.4118
Epoch 219/1500
3/3 [==============================] - 0s 67ms/step - loss: 1.7749 - categorical_accuracy: 0.5176
Epoch 220/1500
3/3 [==============================] - 0s 70ms/step - loss: 2.0360 - categorical_accuracy: 0.4353
Epoch 221/1500
3/3 [==============================] - 0s 69ms/step - loss: 1.4416 - categorical_accuracy: 0.4941
Epoch 222/1500
3/3 [==============================] - 0s 69ms/step - loss: 1.5181 - categorical_accuracy: 0.4118
Epoch 223/1500
3/3 [==============================] - 0s 68ms/step - loss: 1.7792 - categorical_accuracy: 0.4118
Epoch 224/1500
3/3 [==============================] - 0s 63ms/step - loss: 2.5673 - categorical_accuracy: 0.4471
Epoch 225/1500
3/3 [==============================] - 0s 67ms/step - loss: 1.6803 - categorical_accuracy: 0.423

3/3 [==============================] - 0s 67ms/step - loss: 0.8482 - categorical_accuracy: 0.5294
Epoch 290/1500
3/3 [==============================] - 0s 67ms/step - loss: 1.1813 - categorical_accuracy: 0.5176
Epoch 291/1500
3/3 [==============================] - 0s 67ms/step - loss: 1.2242 - categorical_accuracy: 0.5412
Epoch 292/1500
3/3 [==============================] - 0s 68ms/step - loss: 1.9282 - categorical_accuracy: 0.3765
Epoch 293/1500
3/3 [==============================] - 0s 67ms/step - loss: 2.3393 - categorical_accuracy: 0.5294
Epoch 294/1500
3/3 [==============================] - 0s 68ms/step - loss: 1.5227 - categorical_accuracy: 0.4706
Epoch 295/1500
3/3 [==============================] - 0s 65ms/step - loss: 1.2060 - categorical_accuracy: 0.4824
Epoch 296/1500
3/3 [==============================] - 0s 71ms/step - loss: 1.1014 - categorical_accuracy: 0.5176
Epoch 297/1500
3/3 [==============================] - 0s 65ms/step - loss: 1.4876 - categorical_accuracy: 0.352

...

3/3 [==============================] - 0s 73ms/step - loss: 1.0634 - categorical_accuracy: 0.6706
Epoch 1154/1500
3/3 [==============================] - 0s 67ms/step - loss: 1.0484 - categorical_accuracy: 0.6118
Epoch 1155/1500
3/3 [==============================] - 0s 67ms/step - loss: 1.2169 - categorical_accuracy: 0.5059
Epoch 1156/1500
3/3 [==============================] - 0s 64ms/step - loss: 1.1290 - categorical_accuracy: 0.6000
Epoch 1157/1500
3/3 [==============================] - 0s 67ms/step - loss: 0.8880 - categorical_accuracy: 0.6706
Epoch 1158/1500
3/3 [==============================] - 0s 70ms/step - loss: 1.1589 - categorical_accuracy: 0.5294
Epoch 1159/1500
3/3 [==============================] - 0s 69ms/step - loss: 1.0521 - categorical_accuracy: 0.6824
Epoch 1160/1500
3/3 [==============================] - 0s 69ms/step - loss: 0.8960 - categorical_accuracy: 0.6235
Epoch 1161/1500
3/3 [==============================] - 0s 67ms/step - loss: 0.8985 - categorical_accurac

3/3 [==============================] - 0s 67ms/step - loss: 0.7213 - categorical_accuracy: 0.7882
Epoch 1226/1500
3/3 [==============================] - 0s 67ms/step - loss: 0.6578 - categorical_accuracy: 0.7059
Epoch 1227/1500
3/3 [==============================] - 0s 65ms/step - loss: 0.6803 - categorical_accuracy: 0.6706
Epoch 1228/1500
3/3 [==============================] - 0s 69ms/step - loss: 0.9350 - categorical_accuracy: 0.6706
Epoch 1229/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.9380 - categorical_accuracy: 0.6353
Epoch 1230/1500
3/3 [==============================] - 0s 67ms/step - loss: 1.2522 - categorical_accuracy: 0.5765
Epoch 1231/1500
3/3 [==============================] - 0s 70ms/step - loss: 0.7967 - categorical_accuracy: 0.7412
Epoch 1232/1500
3/3 [==============================] - 0s 68ms/step - loss: 0.6988 - categorical_accuracy: 0.7294
Epoch 1233/1500
3/3 [==============================] - 0s 63ms/step - loss: 0.6407 - categorical_accurac

3/3 [==============================] - 0s 68ms/step - loss: 0.5985 - categorical_accuracy: 0.7294
Epoch 1298/1500
3/3 [==============================] - 0s 63ms/step - loss: 0.7053 - categorical_accuracy: 0.7529
Epoch 1299/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.7165 - categorical_accuracy: 0.6824
Epoch 1300/1500
3/3 [==============================] - 0s 68ms/step - loss: 0.8169 - categorical_accuracy: 0.6353
Epoch 1301/1500
3/3 [==============================] - 0s 70ms/step - loss: 0.9393 - categorical_accuracy: 0.6471
Epoch 1302/1500
3/3 [==============================] - 0s 68ms/step - loss: 0.5755 - categorical_accuracy: 0.7647
Epoch 1303/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.7832 - categorical_accuracy: 0.6706
Epoch 1304/1500
3/3 [==============================] - 0s 69ms/step - loss: 0.8475 - categorical_accuracy: 0.6000
Epoch 1305/1500
3/3 [==============================] - 0s 67ms/step - loss: 0.9863 - categorical_accurac

3/3 [==============================] - 0s 67ms/step - loss: 0.6125 - categorical_accuracy: 0.6941
Epoch 1370/1500
3/3 [==============================] - 0s 67ms/step - loss: 0.7773 - categorical_accuracy: 0.7294
Epoch 1371/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.6423 - categorical_accuracy: 0.7647
Epoch 1372/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.6135 - categorical_accuracy: 0.6941
Epoch 1373/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.6389 - categorical_accuracy: 0.7529
Epoch 1374/1500
3/3 [==============================] - 0s 70ms/step - loss: 0.5223 - categorical_accuracy: 0.7647
Epoch 1375/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.6186 - categorical_accuracy: 0.7294
Epoch 1376/1500
3/3 [==============================] - 0s 69ms/step - loss: 0.7537 - categorical_accuracy: 0.6941
Epoch 1377/1500
3/3 [==============================] - 0s 63ms/step - loss: 1.0645 - categorical_accurac

3/3 [==============================] - 0s 69ms/step - loss: 0.5763 - categorical_accuracy: 0.7529
Epoch 1442/1500
3/3 [==============================] - 0s 67ms/step - loss: 0.5615 - categorical_accuracy: 0.7882
Epoch 1443/1500
3/3 [==============================] - 0s 63ms/step - loss: 0.5582 - categorical_accuracy: 0.7412
Epoch 1444/1500
3/3 [==============================] - 0s 69ms/step - loss: 0.6996 - categorical_accuracy: 0.7176
Epoch 1445/1500
3/3 [==============================] - 0s 68ms/step - loss: 0.8230 - categorical_accuracy: 0.7176
Epoch 1446/1500
3/3 [==============================] - 0s 65ms/step - loss: 0.7577 - categorical_accuracy: 0.6588
Epoch 1447/1500
3/3 [==============================] - 0s 72ms/step - loss: 0.7773 - categorical_accuracy: 0.6706
Epoch 1448/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.5022 - categorical_accuracy: 0.7882
Epoch 1449/1500
3/3 [==============================] - 0s 67ms/step - loss: 0.5100 - categorical_accurac

In [109]:
res =[7,0.2,0.1]

In [110]:
actions[np.argmax(res)]

'hello'

In [41]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_16 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_17 (LSTM)              (None, 64)                49408     
                                                                 
 dense_15 (Dense)            (None, 64)                4160      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [42]:
res = model.predict(X_test)

1/1 [==============================] - 0s 245ms/step


In [46]:
actions[np.argmax(res[2])]

'hello'

In [45]:
actions[np.argmax(y_test[2])]

'hello'

In [47]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

In [52]:
model.save("action.h5")



In [195]:
del model

In [53]:
model.load_weights("action.h5")

In [54]:
ythat =model.predict(X_test)

1/1 [==============================] - 0s 24ms/step


In [55]:
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(ythat,axis = 1).tolist()

In [58]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[3, 0],
        [0, 2]],

       [[3, 0],
        [0, 2]],

       [[4, 0],
        [0, 1]]], dtype=int64)

In [57]:
accuracy_score(ytrue,yhat)

1.0

TEST

In [72]:
sequance = []
sentence =[]
threshold = 0.7



cap = cv2.VideoCapture(0)
#acess media pipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence =  0.5) as holistic:
    while cap.isOpened():
        #Read the feed
        ret,frame =cap.read()

        #make detection
        image,results = mediapipe_detection(frame,holistic)
        
        #draw landmarks
        draw_style_landmarks(image,results)
        keypoints = extract_keypoints(results)
        sequance.insert(0,keypoints)
        sequance = sequance[:30]
        
        if len(sequance) == 30:
            res = model.predict(np.expand_dims(sequance,axis = 0))[0]
            print(actions[np.argmax(res)])
        
        #visualzation
        if res[np.argmax(res)] > threshold:
            if len(sentence) >0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])
                    
        if len(sentence) >5:
            sentence = sentence[-5:]
        
        cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(image," ".join(sentence),(3,30),
                   cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        
        
        
         #showw screen 
        cv2.imshow("OpenCV Frrd" , image)

        if cv2.waitKey(10) & 0xFF ==ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 27ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 29ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 21ms/step
iloveyou
1/1 [==============================] - 0s 18ms/step
iloveyou
1/1 [==============================] - 0s 20ms/step
iloveyou
1/1 [==============================] - 0s 12ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [===================

1/1 [==============================] - 0s 15ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 20ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 31ms/step
iloveyou
1/1 [==============================] - 0s 31ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 26ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 21ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [=======================

1/1 [==============================] - 0s 33ms/step
iloveyou
1/1 [==============================] - 0s 27ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 5ms/step
iloveyou
1/1 [==============================] - 0s 31ms/step
iloveyou
1/1 [==============================] - 0s 17ms/step
iloveyou
1/1 [==============================] - 0s 8ms/step
iloveyou
1/1 [==============================] - 0s 20ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 14ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 34ms/step
iloveyou
1/1 [=====================

iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 26ms/step
iloveyou
1/1 [==============================] - 0s 18ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==========

1/1 [==============================] - 0s 18ms/step
iloveyou
1/1 [==============================] - 0s 18ms/step
iloveyou
1/1 [==============================] - 0s 17ms/step
iloveyou
1/1 [==============================] - 0s 34ms/step
iloveyou
1/1 [==============================] - 0s 26ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 33ms/step
iloveyou
1/1 [==============================] - 0s 26ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 21ms/step
iloveyou
1/1 [==============================] - 0s 21ms/step
iloveyou
1/1 [==============================] - 0s 13ms/step
iloveyou
1/1 [==============================] - 0s 20ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 18ms/step
iloveyou
1/1 [===================

1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 8ms/step
hello
1/1 [==============================] - 0s 20ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 28ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 20ms/step
hello
1/1 [==============================] - 0s 24ms/step
hello
1/1 [==============================] - 0s 27ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 31ms/step
hello
1/1 [==============================] - 0s 21ms/step
hello
1/1 [==========

1/1 [==============================] - 0s 31ms/step
hello
1/1 [==============================] - 0s 24ms/step
hello
1/1 [==============================] - 0s 31ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 8ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 21ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 23ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 24ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==========

1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 8ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 22ms/step
hello
1/1 [==============================] - 0s 22ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 21ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 8ms/step
hello
1/1 [==============================] - 0s 8ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 8ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 8ms/step
hello
1/1 [==============

1/1 [==============================] - 0s 31ms/step
hello
1/1 [==============================] - 0s 15ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 13ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 8ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 24ms/step
hello
1/1 [==============================] - 0s 21ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 19ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 22ms/step
hello
1/1 [==========

1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 25ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 12ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 24ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 22ms/step
hello
1/1 [==============================] - 0s 14ms/step
hello
1/1 [==============================] - 0s 8ms/step
hello
1/1 [==============================] - 0s 8ms/step
hello
1/1 [==============================] - 0s 19ms/step
hello
1/1 [==============================] - 0s 19ms/step
hello
1/1 [===========

In [69]:
cap.release()
cv2.destroyAllWindows()